# Biostat 257 Homework 5

## Ian Frankenburg

**Due June 5 @ 11:59PM**

In [ ]:
versioninfo()

Again we continue with the linear mixed effects model (LMM)
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T.
$$
Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2).
$$

In HW4, we used the nonlinear programming (NLP) approach (Newton type algorithms) for optimization. In this assignment, we derive and implement an expectation-maximization (EM) algorithm for the same problem.

In [ ]:
# load necessary packages; make sure install them first
using BenchmarkTools, Distributions, LinearAlgebra, Random, Revise

## Q1. (10 pts) Refresher on normal-normal model

Assume the conditional distribution
$$
\mathbf{y} \mid \boldsymbol{\gamma} \sim N(\mathbf{X} \boldsymbol{\beta} + \mathbf{Z} \boldsymbol{\gamma}, \sigma^2 \mathbf{I}_n)
$$
and the prior distribution
$$
\boldsymbol{\gamma} \sim N(\mathbf{0}_q, \boldsymbol{\Sigma}).
$$
By the Bayes theorem, the posterior distribution is
\begin{eqnarray*}
f(\boldsymbol{\gamma} \mid \mathbf{y}) &=& \frac{f(\mathbf{y} \mid \boldsymbol{\gamma}) \times f(\boldsymbol{\gamma})}{f(\mathbf{y})}, \end{eqnarray*}
where $f$ denotes corresponding density. 

Show that the posterior distribution of random effects $\boldsymbol{\gamma}$ is a multivariate normal with mean
\begin{eqnarray*}
\mathbb{E} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& \sigma^{-2} (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1 } \mathbf{Z}^T (\mathbf{y} - \mathbf{X} \boldsymbol{\beta}) \\
&=& \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} (\mathbf{y} - \mathbf{X} \boldsymbol{\beta})
\end{eqnarray*}
and covariance
\begin{eqnarray*}
\text{Var} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1} \\
&=& \boldsymbol{\Sigma} - \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} \mathbf{Z} \boldsymbol{\Sigma}.
\end{eqnarray*}

## Q2 Solution
The distribution $f(\gamma|y)$ is proportional to $f(y|\gamma)f(\gamma)$. Therefore, 
$$
\begin{align*}
f(\gamma|y)\propto f(y|\gamma)f(\gamma)&=N(y;X\beta+Z\gamma,\sigma^2I)N(\gamma;0,\Sigma)\\
&=\exp\Big(-\frac{1}{2\sigma^2}(y-X\beta-Z\gamma)^\top(y-X\beta-Z\gamma)\Big)\exp\Big(-\frac{1}{2}\gamma^\top\Sigma^{-1}\gamma\Big).
\end{align*}
$$
The crux of this argument will be combining the quadratic forms in the exponentials. I'll start first with what I'm after: From conjugacy, I know the posterior will be Gaussian, so I need the quadratic forms to combine as 
$$
\begin{align}
(\gamma-m)^\top M(\gamma-m)
\end{align}
$$
for some $m$ and some $M$.

Therefore,
$$
\begin{align*}
(\gamma-m)^\top M(\gamma-m)&\equiv (y-X\beta-Z\gamma)^\top(y-X\beta-Z\gamma)/\sigma^2+\gamma^\top\Sigma^{-1}\gamma\\
&=(y-X\beta)^\top(y-X\beta)/\sigma^2+\gamma^\top Z^\top Z\gamma/\sigma^2-2\gamma^\top Z^\top(y-X\beta)/\sigma^2+\gamma^\top\Sigma^{-1}\gamma\\
&\propto\gamma^\top(Z^\top Z/\sigma^2+\Sigma^{-1})\gamma-2\gamma^\top Z^\top(y-X\beta)/\sigma^2.
\end{align*}
$$
Expanding $(\gamma-m)^\top M(\gamma-m)$ and symbol-matching gives
$$
\begin{align*}
M&=Z^\top Z/\sigma^2+\Sigma^{-1}\\
m&=M^{-1}Z^\top(y-X\beta)/\sigma^2.
\end{align*}
$$
After I combine these quadratic forms, I can recognize the kernel of a Gaussian $N(m,M^{-1})$, which exactly matches the moments above after applying the Woodbury identity to $M^{-1}$.

## Q2. (20 pts) Derive EM algorithm

1. Write down the complete log-likelihood
$$
\sum_{i=1}^m \log f(\mathbf{y}_i, \boldsymbol{\gamma}_i \mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)
$$

2. Derive the $Q$ function (E-step).
$$
Q(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2 \mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}).
$$

3. Derive the EM (or ECM) update of $\boldsymbol{\beta}$, $\boldsymbol{\Sigma}$, and $\sigma^2$ (M-step). 

## Q2. Solution
1.
For the $i$-th data point, the model is given as
$$
y_i=X_i\beta+Z_i\gamma_i+\varepsilon_i.
$$
Therefore, I can re-express this as a multivariate linear model through
\begin{align*}
\begin{pmatrix}
y_1\\
\vdots\\
y_m
\end{pmatrix}&=
\begin{pmatrix}
X_1\\
\vdots\\
X_m
\end{pmatrix}
\begin{pmatrix}
\beta\\
\vdots\\
\beta
\end{pmatrix}+
\begin{pmatrix}
Z_1 & 0 & \cdots & 0\\
0 & Z_2 & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & Z_m
\end{pmatrix}\begin{pmatrix}
\gamma_1\\
\vdots\\
\gamma_m
\end{pmatrix}+
\begin{pmatrix}
\varepsilon_1\\
\vdots\\
\varepsilon_m
\end{pmatrix}\\
\Rightarrow\mathbf{y}&=\mathbf{X}\boldsymbol{\beta}+\mathbf{Z}\boldsymbol\gamma+\boldsymbol\varepsilon.
\end{align*}
1. 
To find the joint log-likelihood, I notice
$$
\sum \log f({y}_i, {\gamma}_i \mid {\beta}, {\Sigma}, \sigma^2)=\sum_i\log f({y}_i|{\gamma}_i, {\beta}, {\Sigma}, \sigma^2)+\sum_i\log f(\gamma_i \mid {\Sigma}).
$$

2. 
To derive the expectation-maximum algorithm, I'll start with what I know: the joint log-likelihood is maximized at the quantities $(\hat{\boldsymbol\beta},\hat\sigma^2)$ and $\hat{\boldsymbol\Sigma}$. The MLE $(\hat{\boldsymbol\beta},\hat\sigma^2)$ maximizes the first component of the joint log-likelihood
$$
\begin{align*}
\sum_i\log f({y}_i|{\gamma}_i, {\beta}, {\Sigma}, \sigma^2),
\end{align*}
$$
while the MLE $\hat\Sigma$ maximizes 
$$
\begin{align*}
\sum_i\log f(\gamma_i \mid {\Sigma}).
\end{align*}
$$
This is a normal model, so I know the explicit form the MLEs will take, namely
$$
\begin{align*}
\hat{\boldsymbol{\beta}} &= (\mathbf{X}^\top \mathbf{X})^{-1}\mathbf{X}^\top(\mathbf{y}-\mathbf{Z}\boldsymbol{\gamma})\\
\Rightarrow\hat\beta &= (\sum_{i=1}^m X_i^\top X_i)^{-1}\sum_{i=1}^m X_i^\top(y_i-Z_i\gamma_i)\\
\hat\sigma^2&=\frac{RSS}{\sum_i n_i}=\frac{1}{\sum_i n_i}(\mathbf{y}-\mathbf X\boldsymbol\beta-\mathbf Z\boldsymbol\gamma)^\top(\mathbf{y}-\mathbf X\boldsymbol\beta-\mathbf Z\boldsymbol\gamma)\\
&=\frac{1}{\sum_i n_i}\big[(\mathbf{y}-\mathbf X\boldsymbol\beta)^\top(\mathbf{y}-\mathbf X\boldsymbol\beta)-2\boldsymbol\gamma^\top\mathbf Z^\top(\mathbf y-\mathbf X\boldsymbol\beta)+\boldsymbol\gamma^\top\mathbf Z^\top\mathbf Z\boldsymbol\gamma\big]\\
\hat{\Sigma} &=\frac{1}{m}\sum_{i=1}^m\gamma_i\gamma_i^\top
\end{align*}
$$
Only the last MLE might be non-elementary, so I'll quickly derive it. Start with the log-likelihood for $\Sigma$ and then find $\nabla_\Sigma$ and equate to zero. The log-likelihood for $\Sigma$ is proportional to
$$
\begin{align*}
-\frac{m}{2}\log\det(\Sigma)-\frac{1}{2}\sum_{i=1}^m\gamma_i^\top\Sigma^{-1}\gamma_i.
\end{align*}
$$
The gradient of this is 
$$
\begin{align*}
&\nabla_\Sigma\Big\{-\frac{m}{2}\log\det(\Sigma)-\frac{1}{2}\sum_{i=1}^m\gamma_i^\top\Sigma^{-1}\gamma_i\Big\}\\
=&-\frac{m}{2}\Sigma^{-1}+\frac{1}{2}\sum_{i=1}^m\Sigma^{-1}\gamma_i\gamma_i^\top\Sigma^{-1}:=0\\
\Rightarrow&\hat{\Sigma} = \frac{1}{m}\sum_{i=1}^m\gamma_i\gamma_i^\top.
\end{align*}
$$
By the structure of this multivariate regression, the MLEs $\hat{\boldsymbol\beta}$, $\hat{\boldsymbol\Sigma}$, and $\hat{\sigma}^2$ depend on knowing $\gamma_i$ and $\gamma_i\gamma_i^\top$. The expectation-maximization algorithm uses the expectation $\mathbb E[\gamma_i|y_i,\beta,\sigma^2,\Sigma]$ to get estimates for these quantities and then iteratively updates them.
The expectation $\mathbb E[\gamma_i|y_i]$ is just the mean I calculated in Q1
$$
m_i:=\Sigma {Z_i}^T ({Z_i} {\Sigma} {Z_i}^T + \sigma^2 {I})^{-1} ({y_i} - {X_i} {\beta}),
$$
so I'll use this to fix $\gamma_i$. I still need to fix $\gamma_i\gamma_i^\top$, so I'll use 
$$
\mathbb E[\gamma_i\gamma_i^\top|y_i]=M_i^{-1} + \mathbb E[\gamma_i|y_i]\mathbb E[\gamma_i|y_i]^\top.
$$

3.
Therefore, the maximization step will be
$$
\begin{align*}
\boldsymbol{\beta}_{s+1} &= (\mathbf{X}^\top \mathbf{X})^{-1}\mathbf{X}^\top(\mathbf{y}-\mathbf{Z}\mathbb E[\boldsymbol\gamma|\mathbf y,\boldsymbol{\beta}_{s},\sigma^2_{s},\boldsymbol{\Sigma}_{s}])\\
\sigma^2_{s+1} &= \frac{1}{\sum_i n_i}\big[(\mathbf{y}-\mathbf X\boldsymbol\beta)^\top(\mathbf{y}-\mathbf X\boldsymbol\beta_s)-2(\mathbf y-\mathbf X\boldsymbol\beta_s)^\top\mathbf Z\mathbb E[\boldsymbol\gamma|\mathbf y,\boldsymbol\beta_s,\sigma_s^2,\boldsymbol\Sigma_s]+tr(\mathbf Z^\top\mathbf Z\mathbb E[\boldsymbol{\gamma\gamma}^\top|\mathbf y,\boldsymbol{\beta}_{s},\sigma^2_{s},\boldsymbol{\Sigma}_{s}])\big]\\
&=\frac{1}{\sum_i n_i}\big[(\sum_{i=1}^m({y_i}-X_i\beta_s)^\top(y_i-X_i\beta_s)-2\sum_{i=1}^m(y_i-X_i\beta)^\top Z_i\mathbb E[\gamma_i|y_i]+\sum_{i=1}^m tr(Z_iZ_i^\top\mathbb E[\gamma_i\gamma_i^\top|y_i])\big]\\
\boldsymbol\Sigma_{s+1}&=\sum_{i=n}^n \mathbb E[\gamma_i\gamma_i^\top|y_i,{\beta}_{s},\sigma^2_{s},\Sigma_{s}]
\end{align*}
$$

## Q3. (20 pts) Objective of a single datum

We modify the code from HW4 to evaluate the objective, the conditional mean of $\boldsymbol{\gamma}$, and the conditional variance of $\boldsymbol{\gamma}$. Start-up code is provided below. You do _not_ have to use this code.

In [ ]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # posterior mean and variance of random effects γ
    μγ         :: Vector{T} # posterior mean of random effects
    νγ         :: Matrix{T} # posterior variance of random effects
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    rtr        :: Vector{T}
    xty        :: Vector{T}
    zty        :: Vector{T}
    ztr        :: Vector{T}
    ltztr      :: Vector{T}
    xtr        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    ltztzl     :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2 :: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
    y::Vector{T}, 
    X::Matrix{T}, 
    Z::Matrix{T}) where T <: AbstractFloat
    n, p, q = size(X, 1), size(X, 2), size(Z, 2)
    μγ         = Vector{T}(undef, q)
    νγ         = Matrix{T}(undef, q, q)
    yty        = abs2(norm(y))
    rtr        = Vector{T}(undef, 1)
    xty        = transpose(X) * y
    zty        = transpose(Z) * y
    ztr        = similar(zty)
    ltztr      = similar(zty)
    xtr        = Vector{T}(undef, p)
    storage_p  = similar(xtr)
    storage_q  = Vector{T}(undef, q)
    storage_q2  = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    ltztzl     = similar(ztz)
    storage_qq = similar(ztz)
    storage_qq2 = similar(ztz)

    LmmObs(y, X, Z, μγ, νγ, 
        yty, rtr, xty, zty, ztr, ltztr, xtr,
        storage_p, storage_q, storage_q2, 
        xtx, ztx, ztz, ltztzl, storage_qq, storage_qq2)
end

"""
    logl!(obs::LmmObs, β, Σ, L, σ², updater = false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `Σ`, 
and `σ²`. The lower triangular Cholesky factor `L` of `Σ` must be supplied too.
The fields `obs.μγ` and `obs.νγ` are overwritten by the posterior mean and 
posterior variance of random effects. If `updater==true`, fields `obs.ztr`, 
`obs.xtr`, and `obs.rtr` are updated according to input parameter values. 
Otherwise, it assumes these three fields are pre-computed. 
"""
function logl!(
        obs     :: LmmObs{T}, 
        β       :: Vector{T}, 
        Σ       :: Matrix{T},
        L       :: Matrix{T},
        σ²      :: T,
        updater :: Bool = false
        ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    σ²inv   = inv(σ²)
    ####################
    # Evaluate objective
    ####################
    # form the q-by-q matrix: Lt Zt Z L
    copy!(obs.ltztzl, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.ltztzl) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.ltztzl) # O(q^3)        
    # form the q-by-q matrix: M = σ² I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ltztzl)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # Zt * res
    updater && BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.ztr, obs.zty)) # O(pq)
    # Lt * (Zt * res)
    BLAS.trmv!('L', 'T', 'N', L, copy!(obs.ltztr, obs.ztr))    # O(q^2)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, copy!(obs.storage_q, obs.ltztr)) # O(q^3)
    # Xt * res = Xt * y - Xt * X * β
    updater && BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), copy!(obs.xtr, obs.xty))
    # l2 norm of residual vector
    updater && (obs.rtr[1] = obs.yty - dot(obs.xty, β) - dot(obs.xtr, β))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q # log det term
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (obs.rtr[1] - qf) * σ²inv 
    logl /= -2
    ######################################
    # TODO: Evaluate posterior mean and variance
    ###################################### 
    
    copy!(obs.storage_qq2, L)
    LAPACK.trtri!('L', 'N', obs.storage_qq2) # invert cholesky factor of Σ
    BLAS.gemm!('T','N',T(1),obs.storage_qq2, obs.storage_qq2, T(0), obs.storage_qq) # get full Σ⁻¹
    BLAS.axpy!(1/σ², obs.ztz, obs.storage_qq) # add Z'Z/σ² to Σ⁻¹
    
    copy!(obs.storage_qq2, obs.storage_qq)
    LAPACK.potrf!('U',obs.storage_qq2) # Cholesky decompose Z'Z/σ² + Σ⁻¹ as UᵀU
    LAPACK.potri!('U', obs.storage_qq2) # invert cholesky decomposition
    LinearAlgebra.copytri!(obs.storage_qq2, 'U')

    BLAS.gemm!('N','N',T(1),obs.storage_qq2, obs.ztr, T(0), obs.storage_q2) # get M⁻¹Z'r
    obs.storage_q2 ./= σ²
    
 
    obs.μγ .= obs.storage_q2
    obs.νγ .= obs.storage_qq2
    ###################
    # Return
    ###################        
    return logl
end


It is a good idea to test correctness and efficiency of the single datum objective/posterior mean/var evaluator here. It's the same test datum in HW2 and HW4.

In [4]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X = [ones(n) randn(n, p - 1)]
Z = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
#L = Matrix(Symmetric(Σ))
#LAPACK.potrf!('L', L)
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [5]:
@show logl = logl!(obs, β, Σ, L, σ², true)
@show obs.μγ
@show obs.νγ;

logl = logl!(obs, β, Σ, L, σ², true) = -3247.4568580638243
obs.μγ = [-1.7352999248283547, -1.223466577704898, -0.2502019040776347]
obs.νγ = [0.0007495521480103862 4.1880268195223775e-6 8.59502834901115e-6; 4.1880268195223775e-6 0.0007599372708603274 -1.0092121486077335e-5; 8.59502834901115e-6 -1.0092121486077335e-5 0.0007370698232610101]


You will lose all 20 points if following statement throws `AssertionError`.

In [6]:
@assert abs(logl - (-3247.4568580638247)) < 1e-8
@assert norm(obs.μγ - [-1.7352999248278138, 
        -1.2234665777052611, -0.25020190407767146]) < 1e-8
@assert norm(obs.νγ - [0.0007495521482876466 4.188026899159083e-6 8.595028393969659e-6; 
        4.1880268803062436e-6 0.0007599372708508531 -1.0092121451703577e-5; 
        8.595028373480989e-6 -1.009212147054782e-5 0.0007370698230021235]) < 1e-8

### Efficiency

Benchmark for efficiency.

In [7]:
bm_obj = @benchmark logl!($obs, $β, $Σ, $L, $σ², true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.556 μs (0.00% GC)
  median time:      1.569 μs (0.00% GC)
  mean time:        1.626 μs (0.00% GC)
  maximum time:     5.554 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

My median runt time is 1.7μs. You will get full credit if the median run time is within 10μs. The points you will get are

In [8]:
clamp(10 / (median(bm_obj).time / 1e3) * 10, 0, 10)

10.0

In [9]:
# # check for type stability
# @code_warntype logl!(obs, β, Σ, L, σ²)

In [10]:
# using Profile

# Profile.clear()
# @profile for i in 1:10000; logl!(obs, β, Σ, L, σ²); end
# Profile.print(format=:flat)

## Q4. LmmModel type

We modify the `LmmModel` type in HW4 to hold all data points, model parameters, and intermediate arrays.

In [11]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat}
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β      :: Vector{T}
    Σ      :: Matrix{T}
    L      :: Matrix{T}
    σ²     :: Vector{T}    
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty    :: Vector{T}
    xtr    :: Vector{T}
    ztr2   :: Vector{T}
    xtxinv :: Matrix{T}
    ztz2   :: Matrix{T}
    storage_p   :: Vector{T}
    storage_q   :: Vector{T}
    storage_qq   :: Matrix{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p      = size(obsvec[1].X, 2)
    q      = size(obsvec[1].Z, 2)
    # parameters
    β      = Vector{T}(undef, p)
    Σ      = Matrix{T}(undef, q, q)
    L      = Matrix{T}(undef, q, q)
    σ²     = Vector{T}(undef, 1)    
    # intermediate arrays
    xty    = zeros(T, p)
    xtr    = similar(xty)
    ztr2   = Vector{T}(undef, abs2(q))
    xtxinv = zeros(T, p, p)
    storage_p = similar(xty)
    storage_q = Vector{T}(undef, q) 
    storage_qq = similar(L)
    # pre-calculate \sum_i Xi^T Xi and \sum_i Xi^T y_i
    @inbounds for i in eachindex(obsvec)
        obs = obsvec[i]
        BLAS.axpy!(T(1), obs.xtx, xtxinv)
        BLAS.axpy!(T(1), obs.xty, xty)
    end
    # invert X'X
    LAPACK.potrf!('U', xtxinv)
    LAPACK.potri!('U', xtxinv)
    LinearAlgebra.copytri!(xtxinv, 'U')
    ztz2   = Matrix{T}(undef, abs2(q), abs2(q))
    LmmModel(obsvec, β, Σ, L, σ², xty, xtr, ztr2, xtxinv,
        ztz2,storage_p, storage_q, storage_qq)
end

LmmModel

## Q5. Implement EM update

Let's write the key function `update_em!` that performs one iteration of EM update.

In [58]:
"""
    update_em!(m::LmmModel, updater::Bool = false)

Perform one iteration of EM update. It returns the log-likelihood calculated 
from input `m.β`, `m.Σ`, `m.L`, and `m.σ²`. These fields are then overwritten 
by the next EM iterate. The fields `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` are updated according to the resultant `m.β`. If `updater==true`, 
the function first updates `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` according to `m.β`. If `updater==false`, it assumes these fields 
are pre-computed.
"""
function update_em!(m::LmmModel{T}, updater::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    n = length(m.data)
    # TODO: update m.β
    # m.β = (Xᵢ'Xᵢ)⁻¹Xᵢ'(yᵢ-Zᵢmᵢ)
    fill!(m.storage_p, 0)
    @inbounds for i in 1:n
        logl += logl!(m.data[i], m.β, m.Σ, m.L, m.σ²[1], updater)
        BLAS.gemv!('T', T(-1), m.data[i].ztx, m.data[i].μγ, T(1), m.storage_p)
    end
    BLAS.axpy!(T(1), m.xty, m.storage_p)
    BLAS.gemv!('N', T(1), m.xtxinv, m.storage_p, T(0), m.β)
    # TODO: update m.data[i].ztr, m.data[i].xtr, m.data[i].rtr
    @inbounds for i in 1:n
        BLAS.gemv!('N', T(-1), m.data[i].ztx, m.β, T(1), copy!(m.data[i].ztr, m.data[i].zty))
        BLAS.gemv!('N', T(-1), m.data[i].xtx, m.β, T(1), copy!(m.data[i].xtr, m.data[i].xty))
        m.data[i].rtr[1] = m.data[i].yty - dot(m.data[i].xty, m.β) - dot(m.data[i].xtr, m.β)
    end
    # TODO: update m.σ²
    fill!(m.Σ, 0);nᵢ = 0
    @inbounds for i in 1:n
        m.σ²[1] += m.data[i].rtr[1]
        m.σ²[1] += dot(m.data[i].ztz, m.data[i].νγ) #tr(Z'ZEγγ'|y)
        BLAS.gemv!('N', T(1), m.data[i].ztz, m.data[i].μγ, T(0), m.storage_q)
        BLAS.axpy!(T(-2), m.data[i].ztr, m.storage_q) # cross term
        m.σ²[1] += dot(m.storage_q, m.data[i].μγ)
        nᵢ += size(m.data[i].X, 1)
        # update m.Σ
        BLAS.axpy!(T(1), m.data[i].νγ, m.Σ) 
        BLAS.gemm!('N', 'T', T(1), m.data[i].μγ, m.data[i].μγ, T(1), m.Σ)
    end
    m.σ²[1] /= nᵢ
    m.Σ ./= length(m.data)
    copy!(m.L, m.Σ) # cholesky decompose Σ
    LAPACK.potrf!('L', m.L)
    # return log-likelihood at input parameter values
    logl
end

update_em!

## Q6. (30 pts) Test data

Let's generate a fake longitudinal data set (same as HW4) to test our algorithm.

In [59]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

### Correctness

Evaluate log-likelihood and gradient at the true parameter values.

In [60]:
copy!(lmm.β, βtrue)
copy!(lmm.Σ, Σtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj1 = update_em!(lmm, true)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²
println()
@show obj2 = update_em!(lmm, false)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

obj1 = update_em!(lmm, true) = -2.854712648683302e6
lmm.β = [0.0999926726279833, 6.5001444038353, -3.498793255670913, 1.0004987618552164, 5.000049480766921]
lmm.Σ = [2.0571260191161262 -0.010005181563164128 0.018147868730395873; -0.010005181563164128 1.1823115059222267 -0.022667932474031077; 0.018147868730395873 -0.022667932474031077 0.9380104634208111]
lmm.L = [1.4342684613126395 -0.010005181563164128 0.018147868730395873; -0.006975808109178813 1.087319108644491 -0.022667932474031077; 0.012653048728260386 -0.020766366611781186 0.9682040703263111]
lmm.σ² = [1.4990451246301146]

obj2 = update_em!(lmm, false) = -2.8547098610628443e6
lmm.β = [0.09998548093054863, 6.500144687707346, -3.4987912715251057, 1.0004995336389912, 5.000049506702866]
lmm.Σ = [2.057172301212479 -0.00999843658826873 0.018139919876959325; -0.00999843658826873 1.1822846711285173 -0.02266381741245616; 0.018139919876959325 -0.02266381741245616 0.9379040474052037]
lmm.L = [1.4342845956129067 -0.00999843658826873 0.0181399

1-element Array{Float64,1}:
 1.4990435336709524

Test correctness. You will loss all 30 points if following code throws `AssertError`.

In [61]:
@assert abs(obj1 - (-2.854712648683302e6)) < 1e-6
@assert abs(obj2 - (-2.8547098610621705e6)) < 1e-6

### Efficiency

Test efficiency of EM update.

In [62]:
bm_emupdate = @benchmark update_em!($lmm, true) setup=(
    copy!(lmm.β, βtrue);
    copy!(lmm.Σ, Σtrue);
    copy!(lmm.L, Ltrue);
    lmm.σ²[1] = σ²true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.029 ms (0.00% GC)
  median time:      2.287 ms (0.00% GC)
  mean time:        2.341 ms (0.00% GC)
  maximum time:     5.615 ms (0.00% GC)
  --------------
  samples:          2130
  evals/sample:     1

My median run time is 2.17ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [63]:
clamp(10 / (median(bm_emupdate).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [64]:
clamp(10 - median(bm_emupdate).memory / 100, 0, 10)

10.0

## Q7. Starting point

We use the same least squares estimates as in HW4 as starting point. 

In [65]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # LS estimate for β
    mul!(m.β, m.xtxinv, m.xty)
    # LS etimate for σ2 and Σ
    rss, ntotal = zero(T), 0
    fill!(m.ztz2, 0)
    fill!(m.ztr2, 0)    
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        ntotal += length(obs.y)
        # update Xt * res
        BLAS.gemv!('N', T(-1), obs.xtx, m.β, T(1), copy!(obs.xtr, obs.xty))
        # rss of i-th individual
        rss += obs.yty - dot(obs.xty, m.β) - dot(obs.xtr, m.β)
        # update Zi' * res
        BLAS.gemv!('N', T(-1), obs.ztx, m.β, T(1), copy!(obs.ztr, obs.zty))
        # Zi'Zi ⊗ Zi'Zi
        kron_axpy!(obs.ztz, obs.ztz, m.ztz2)
        # Zi'res ⊗ Zi'res
        kron_axpy!(obs.ztr, obs.ztr, m.ztr2)
    end
    m.σ²[1] = rss / ntotal
    # LS estimate for Σ = LLt
    LAPACK.potrf!('U', m.ztz2)
    BLAS.trsv!('U', 'T', 'N', m.ztz2, m.ztr2)
    BLAS.trsv!('U', 'N', 'N', m.ztz2, m.ztr2)
    copyto!(m.Σ, m.ztr2)
    copy!(m.L, m.Σ)
    LAPACK.potrf!('L', m.L)
    for j in 2:q, i in 1:j-1
        m.L[i, j] = 0
    end
    m
end

"""
    kron_axpy!(A, X, Y)

Overwrite `Y` with `A ⊗ X + Y`. Same as `Y += kron(A, X)` but
more memory efficient.
"""
function kron_axpy!(
        A::AbstractVecOrMat{T},
        X::AbstractVecOrMat{T},
        Y::AbstractVecOrMat{T}
        ) where T <: Real
    m, n = size(A, 1), size(A, 2)
    p, q = size(X, 1), size(X, 2)
    @assert size(Y, 1) == m * p
    @assert size(Y, 2) == n * q
    @inbounds for j in 1:n
        coffset = (j - 1) * q
        for i in 1:m
            a = A[i, j]
            roffset = (i - 1) * p            
            for l in 1:q
                r = roffset + 1
                c = coffset + l
                for k in 1:p                
                    Y[r, c] += a * X[k, l]
                    r += 1
                end
            end
        end
    end
    Y
end

kron_axpy!

In [66]:
init_ls!(lmm)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

lmm.β = [0.08757868042872417, 6.498740647737429, -3.497052864661033, 0.9971817478195715, 5.000992653697893]
lmm.Σ = [2.04716308425845 -0.002995972291969754 0.01891179766062129; -0.0029959722919697535 1.187049602755533 -0.012600387416032; 0.018911797660621284 -0.012600387416032 0.9228737560869614]
lmm.L = [1.4307910693942878 0.0 0.0; -0.0020939271680232604 1.08951604771318 0.0; 0.013217721346714457 -0.0115397203158173 0.9605029322102014]
lmm.σ² = [5.667244661294349]


1-element Array{Float64,1}:
 5.667244661294349

## Q8. Estimation by EM

We write a function `fit!` that implements the EM algorithm for estimating LMM.

In [67]:
"""
    fit!(m::LmmModel)

Fit an `LmmModel` object by MLE using a EM algorithm. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m       :: LmmModel;
        maxiter :: Integer       = 10_000,
        ftolrel :: AbstractFloat = 1e-12,
        prtfreq :: Integer       = 0
    )
    obj = update_em!(m, true)
    for iter in 0:maxiter
        obj_old = obj
        # EM update
        obj = update_em!(m, false)
        # print obj
        prtfreq > 0 && rem(iter, prtfreq) == 0 && println("iter=$iter, obj=$obj")
        # check monotonicity
        obj < obj_old && (@warn "monotoniciy violated")
        # check convergence criterion
        (obj - obj_old) < ftolrel * (abs(obj_old) + 1) && break
        # warning about non-convergence
        iter == maxiter && (@warn "maximum iterations reached")
    end
    m
end


fit!

## Q9. (20 pts) Test drive

Now we can run our EM algorithm to compute the MLE.

In [68]:
# initialize from least squares
init_ls!(lmm)

@time fit!(lmm, prtfreq = 100);

println("objective value at solution: ", update_em!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)

iter=0, obj=-2.854719643876485e6
iter=100, obj=-2.8547097914012955e6
iter=200, obj=-2.854709790941533e6
iter=300, obj=-2.8547097905171304e6
iter=400, obj=-2.8547097901253332e6
iter=500, obj=-2.8547097897635843e6
iter=600, obj=-2.854709789429535e6
iter=700, obj=-2.8547097891210373e6
  1.692702 seconds (135.66 k allocations: 6.432 MiB)
objective value at solution: -2.8547097889785822e6

solution values:
lmm.β = [0.08626244702779734, 6.5001446884543475, -3.4987912684801077, 1.0004995348298902, 5.000049506866046]
lmm.σ² = [1.499043531080369]
lmm.L * transpose(lmm.L) = [2.05731930374695 -0.021106417085553067 0.03636835309503711; -0.021106417085553067 1.1827982894067757 -0.04460571983146433; 0.03636835309503711 -0.04460571983146433 0.9379038237762335]


3×3 Array{Float64,2}:
  2.05732    -0.0211064   0.0363684
 -0.0211064   1.1828     -0.0446057
  0.0363684  -0.0446057   0.937904

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [69]:
# objective at solution should be close enough to the optimal
@assert update_em!(lmm) > -2.85471e6

### Efficiency

My median run time 1.8s. You get 10 points if your median run time is within 5s.

In [70]:
bm_em = @benchmark fit!($lmm) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.452 s (0.00% GC)
  median time:      1.456 s (0.00% GC)
  mean time:        1.456 s (0.00% GC)
  maximum time:     1.460 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1

In [71]:
# this is the points you get
clamp(5 / (median(bm_em).time / 1e9) * 10, 0, 10)

10.0

## Q10. (10 pts) EM vs Newton type algorithms

Contrast EM algorithm to the Newton type algorithms (gradient free, gradient based, using Hessian) in HW4, in terms of the stability, convergence rate (how fast the algorithm is converging),  final objective value, total run time, derivation, and implementation efforts. 

Conecptually, I think the EM algorithm makes more sense to me than the optimization used in HW4, and I found this reflected in my implementation efforts (last homework took me much longer to code). However the EM runtime is much slower, of similar order to lem4 at the end of HW4. Gradient-based methods were much faster, and every method had (expectedly) very similar answers. I like probability calculus, so I prefere the EM algorithm :)